In [2]:
#importing libraries
import h5py
import scipy.io as io
import PIL.Image as Image
import numpy as np
import os
import glob
from matplotlib import pyplot as plt
from scipy.ndimage.filters import gaussian_filter
import scipy
import json
import torchvision.transforms.functional as F
from matplotlib import cm as CM
from image import *
from model import CSRNet
import torch
from tqdm import tqdm
%matplotlib inline

In [3]:
from torchvision import datasets, transforms
transform=transforms.Compose([
                      transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                    std=[0.229, 0.224, 0.225]),
                  ])


In [4]:
#defining the location of dataset
root = r"W:\_00_ Projects\# Pycharm\TES Beta # Machine Learning\Final_Amigo"

part_A_train = os.path.join(root,'Images','Dataset','part_A','train_data','images')
part_A_test = os.path.join(root,'Images','Dataset','part_A','test_data','images')
part_B_train = os.path.join(root,'Images','Dataset','part_B','train_data','images')
part_B_test = os.path.join(root,'Images','Dataset','part_B','test_data','images')
path_sets = [part_A_train,part_A_test]

In [5]:
img_paths = []
for path in path_sets:
    for img_path in glob.glob(os.path.join(path, '*.jpg')):
       img_paths.append(img_path)

In [6]:
model = CSRNet()

In [8]:
#defining the model
model = model.cuda()

In [9]:
#loading the trained weights
checkpoint = torch.load('Images/Dataset/0model_best.pth.tar')
model.load_state_dict(checkpoint['state_dict'])

<All keys matched successfully>

In [10]:
mae = 0
for i in tqdm(range(len(img_paths))):
    img = transform(Image.open(img_paths[i]).convert('RGB')).cuda()
    gt_file = h5py.File(img_paths[i].replace('.jpg','.h5').replace('images','ground-truth-h5'),'r')
    groundtruth = np.asarray(gt_file['density'])
    output = model(img.unsqueeze(0))
    mae += abs(output.detach().cpu().sum().numpy()-np.sum(groundtruth))
print (mae/len(img_paths))

  0%|          | 0/482 [00:00<?, ?it/s]C:\Users\mitul\anaconda3\envs\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
  0%|          | 0/482 [00:08<?, ?it/s]


RuntimeError: CUDA out of memory. Tried to allocate 48.00 MiB (GPU 0; 2.00 GiB total capacity; 1.06 GiB already allocated; 6.88 MiB free; 1.09 GiB reserved in total by PyTorch)

In [15]:
from matplotlib import cm as c
img = transform(Image.open('Images\\Dataset\part_A\\test_data\\images\\IMG_95.jpg'))

output = model(img.unsqueeze(0))
print("Predicted Count : ",int(output.detach().cpu().sum().numpy()))
temp = np.asarray(output.detach().cpu().reshape(output.detach().cpu().shape[2],output.detach().cpu().shape[3]))
plt.imshow(temp,cmap = c.jet)
plt.show()
temp = h5py.File('W:\_00_ Projects\# Pycharm\TES Beta # Machine Learning\Final_Amigo\Images\Dataset\part_A\test_data\Ground-truth-h5\IMG_95.h5', 'r')
temp_1 = np.asarray(temp['density'])
plt.imshow(temp_1,cmap = c.jet)
print("Original Count : ",int(np.sum(temp_1)) + 1)
plt.show()
print("Original Image")
plt.imshow(plt.imread('part_A/test_data/images/IMG_100.jpg'))
plt.show()

RuntimeError: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same or input should be a MKLDNN tensor and weight is a dense tensor